# Fashion MNIST
Fashion Mnist is a Dataset created by Zolando Fashion Wear to replace the Original Mnist and at the same time increasing the difficulty.
This code post is all about how to create a NN model to classify fashion mnist images.

Let’s look at the code.

# 1. Import the Libraries

In [ ]:
#importing the libraries
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 2. Load the Dataset
Pytorch’s torchvision contains inbuilt datasets for practice, such as MNIST, FashionMnist, etc.

If the dataset you want to use is NOT within the torchvision library, you will need to implement your own custom dataset. Here is a tutorial on that: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [ ]:
#load datatset

train_set = torchvision.datasets.FashionMNIST(root = './data/FashionMNIST', download = True,
                                              train = True, transform = transforms.Compose([transforms.ToTensor(),]))

test_set = torchvision.datasets.FashionMNIST(root = './data/FashionMNIST', download=True,
                                             train=False, transform = transforms.Compose([transforms.ToTensor()]))

Note the parameter `transform`. This receives a list of transformations that needs to be done on the input image as a part of pre-processing (such as normalization/standardization, resize, etc) and can be implemented using `transforms.compose`. This can also be used to introduce some data augmentation in the training pipeline.
Please, check here for more details: https://pytorch.org/vision/stable/transforms.html



`Dataset` (above) class in Pytorch basically covers the data in a tuple and enables us to access the index of each data. It is necessary to create `Dataloader` (below) which can be used to shuffle, apply Mini-Batch Gradient Descent and more.

We can also show some figures for visualization.

In [ ]:
# dataloader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=20)  # this is actually reponsible for returning the mini-batch

test_loader = torch.utils.data.DataLoader(test_set, batch_size=60000)

# iterate the train data with samples
images, labels = next(iter(train_loader))

#used to create a grid of images
grid = torchvision.utils.make_grid(images,nrow=20)
plt.figure(figsize=(15,15))
plt.imshow(np.transpose(grid,(1,2,0)),cmap='gray')

# 3. Define the Neural Network Model
Here, we define the NeuralNetwork as the same as in Lab session **4.2**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# define the model
model = NeuralNetwork()

Note: we just define the simple network model with linear function, you can implement any other structures based on the Pytorch framework as we will see in other praticals.

# 4. Define Loss and Optimizer

Define the loss (in this case Cross-Entropy loss) and the optimizer (SGD with momentum) and their hyper-parameters, such as number of epochs, and learning rate.




In [ ]:
# define learning rate
learning_rate = 0.005
# define your optimizer with SGD and learning rate
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
# define the loss function
criterion = nn.CrossEntropyLoss()
# set the epoch
epochs = 15

# 5. Train the network
When all things are ready, we can train the network now.

When feeding dataset into the network , we calculate the loss and use the gradient to update the weights by minimizing the loss function.
We just simply repeat the process several times.

Remember that, if you are using Google Colab, you can change it to use GPUs.

In [ ]:
# loop over the dataset multiple times
losses = []

for i in range(epochs):
  for j, (images,targets) in enumerate(train_loader):

    # making predictions
    y_pred = model(images)

    # calculating loss
    loss = criterion(y_pred,targets.reshape(-1))
    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if i>10:
    optimizer.lr = 0.0005  # reducing the loss to better adjust the dataset
  print(loss)
  losses.append(loss)

print('Complete Training')

# 6. Making a Prediction for A Sample from Test Set



We have trained the network over the training dataset.
Now, we can check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.
Hence, we can calculate the accuracy of the model on the test data.

For the details, let us display an image from the test data firstly.


In [ ]:
# make a prediction for a sample randomly chose from the test dataset
# you can run this part repeatedly, see the classification result from the model
import random
rand_no = random.randint(0,10000)
print(rand_no)

x_test, y_test = next(iter(test_loader))  # getting the test data

y_pred = (model(x_test).argmax(dim=1))  # predicting

label_list = "T-shirt/Top Trouser PullOver Dress Coat Sandal Shirt Sneaker Bag AnkleBoot".split()

plt.imshow(x_test[rand_no].reshape(28,28),cmap='gray')

pred = model(x_test[rand_no].reshape(-1,1,28,28)).argmax()

print("This is a/an {}".format(label_list[pred]))

# 7. Model Performance Analysis
Let us look at how the network performs on the whole dataset.

In [ ]:
# compute the accuracy of the model

print("Accuracy is : ",(y_pred.eq(y_test).sum()/10000.).item()*100,"%")

# 8. Plots loss error

We can show the loss information for each epoch.




In [ ]:
# plot loss figure

lossess = torch.stack(losses, 0)
lossess = lossess.detach().numpy()
plt.plot(lossess)
plt.xlabel('epochs')
plt.ylabel('loss')

<b>Note:</b>
We use the line plots to show epochs along the ``x-axis`` and the loss error on the ``y-axis``.
These plots are sometimes called learning curves. These plots can help to diagnose whether the model has over learned, under learned, or is suitably fit to the training dataset.


# Conclusion
The model’s accuracy is found to be 86% on the test set.
This proves how strong the neural networks are in predicting images.
This is because of the parameter sharing ability and edge-detection ability of network layers.
